In [51]:
import pandas as pd
import joblib

test_df = pd.read_csv("../../tests/data/user/full_features.csv")

model = joblib.load("../../models/gb_model_selected_features_transform.pkl")

import numpy as np
log1p_cols = [
    'gravity_cscba_stddev3x3',
    'mag_uc_2_4km_thd',
    'mag_uc_2_4km_stddev3x3',
    'radio_th_k_ratio',
]

for col in log1p_cols:
    test_df[col] = pd.to_numeric(test_df[col], errors="coerce")
    test_df[f"{col}_log1p"] = np.log1p(test_df[col].clip(lower=0).fillna(0))

test_df.drop(columns=log1p_cols, inplace=True)
X = test_df.drop(columns=["latitude", "longitude"])
for col in X.columns:
    X[col] = X[col].transform(lambda x: x.fillna(x.median()))

## 1.staged_predict_proba() to calculate the 

In [ ]:
import numpy as np
probas = list(model.staged_predict_proba(X))
class1_probs = np.array([p[:, 1] for p in probas])  # shape: (n_stages, n_samples)
uncertainty = class1_probs.std(axis=0)  # shape: (n_samples,)
mean_preds = class1_probs[-1]


In [ ]:
from scipy.stats import norm

confidence_level = 0.95
z_value = norm.ppf((1 + confidence_level) / 2)

confidence_interval = [
    mean_preds - z_value * uncertainty,
    mean_preds + z_value * uncertainty
]
confidence_interval = np.array(confidence_interval).T  

In [58]:
confidence_interval

array([[-0.29662743,  0.29662746],
       [-0.24322034,  0.24322056],
       [-0.09270956,  0.09270956],
       ...,
       [-0.10271963,  0.10272195],
       [-0.22818553,  0.22818611],
       [-0.2089066 ,  0.20900814]])

In [59]:
confidence_interval = pd.DataFrame(confidence_interval, columns=["lower_bound", "upper_bound"])

In [60]:
mean_preds = pd.DataFrame(mean_preds, columns=["mean_preds"])
uncertainty = pd.DataFrame(uncertainty, columns=["uncertainty"])

In [61]:
mean_preds.head()

,mean_preds
0,1.295402e-08
1,1.126062e-07
2,6.810230e-10
3,8.600305e-10
4,8.844805e-06


In [48]:
confidence_interval.head()

,lower_bound,upper_bound
0,-0.296627,0.296627
1,-0.243220,0.243221
2,-0.092710,0.092710
3,-0.093279,0.093279
4,-0.197317,0.197335


In [50]:
test_df = pd.concat([test_df, mean_preds, uncertainty, confidence_interval], axis=1)
test_df.head()

,latitude,longitude,gravity_iso_residual,gravity_cscba,gravity_cscba_1vd,mag_uc_4_8km,radio_k_pct,radio_u_ppm,gravity_cscba_stddev3x3_log1p,mag_uc_2_4km_thd_log1p,mag_uc_2_4km_stddev3x3_log1p,radio_th_k_ratio_log1p,mean_preds,uncertainty,lower_bound,upper_bound
0,-2.347901e+06,7.890331e+05,313.891357,-1711.845459,48.904354,-110.120186,2.089952,1.650577,5.411213,0.010352,0.238804,2.003972,1.295402e-08,0.151343,-0.296627,0.296627
1,-1.327772e+06,1.159960e+06,342.139557,NaN,313.095428,-5.648541,0.110827,1.865251,0.000000,0.000000,0.000000,3.730636,1.126062e-07,0.124094,-0.243220,0.243221
2,-2.825754e+06,1.270949e+06,-221.139679,-708.793701,-477.473206,3.603062,0.134448,1.176058,2.055688,0.007077,0.166527,3.860528,6.810230e-10,0.047302,-0.092710,0.092710
3,-3.175708e+06,1.440866e+06,-99.279518,-759.444763,-237.948151,-2.129511,0.820230,1.127624,2.659562,0.002430,0.060696,2.455139,8.600305e-10,0.047592,-0.093279,0.093279
4,-2.553178e+06,1.751907e+06,424.947296,143.522614,292.836273,20.866171,1.099405,2.181279,3.196141,0.030217,0.579596,2.560828,8.844805e-06,0.100678,-0.197317,0.197335
